In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from IPython.display import Image
import pydotplus
import collections
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data/cleaned_data.csv')
data.head()

,sample_pk,commod,commtype,lab,pestcode,lod,avg_detect,extract,determin,concentration,target
0,239,AJ,RE,WA1,083,0.005,O,805,35,80000.0,0
1,1183,BR,FR,FL1,AFU,0.010,O,805,52,110000.0,1
2,1196,BR,FR,FL1,AFU,0.010,O,805,52,130000.0,1
3,1230,BR,FR,FL1,144,0.005,O,805,35,350000.0,1
4,1269,BR,FR,FL1,180,0.010,O,805,52,260000.0,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 11 columns):
sample_pk        4887 non-null int64
commod           4887 non-null object
commtype         4887 non-null object
lab              4887 non-null object
pestcode         4887 non-null object
lod              4887 non-null float64
avg_detect       4887 non-null object
extract          4887 non-null int64
determin         4887 non-null int64
concentration    4887 non-null float64
target           4887 non-null int64
dtypes: float64(2), int64(4), object(5)
memory usage: 420.1+ KB


In [4]:
[(col, data[col].nunique()) for col in data.columns if data[col].dtype == 'O']

[('commod', 20),
 ('commtype', 4),
 ('lab', 9),
 ('pestcode', 149),
 ('avg_detect', 2)]

In [5]:
data['pestcode'].unique()

array(['083', 'AFU', '144', '180', '512', '114', '666', 'A58', '540',
       '597', '930', 'AFX', '910', 'ABB', '222', '223', 'AEL', '070',
       'AGW', '160', '170', '612', '264', '151', 'B48', 'B58', 'B80',
       '967', '906', '129', '204', 'AHT', 'AGG', 'B79', 'AES', '679',
       'AFO', '249', 'B16', '134', 'B43', 'B77', 'B52', '159', '171',
       '178', '052', '607', 'B75', 'B64', 'AHM', '626', '230', 'AGX',
       '283', 'AEP', 'AGY', '900', '901', '908', '537', '351', 'A30',
       'AGA', 'B61', '907', '028', 'AKD', '902', '024', '200', '014',
       'AFC', '165', '911', '945', '387', '781', '808', '157', 'B56',
       'B22', 'B23', '604', 'AEM', 'B41', 'ADG', '042', 'B85', '956',
       '714', 'B57', 'AKW', 'AJP', 'B10', '651', '237', '699', '382',
       '720', 'AFS', 'B84', 'AHS', '539', '125', 'B82', '948', 'B32',
       'AGT', '624', '026', '731', 'ACG', '633', '333', '780', 'ABN',
       'ACB', 'AEY', '785', 'AED', 'AFB', '032', 'ACH', '305', '784',
       '964', '149',

In [6]:
categoricals = ['commod', 'commtype', 'lab', 'pestcode', 'avg_detect']

In [8]:
X = data.drop('target', axis=1)
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25565, test_size=.5)

In [9]:
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
encoder.fit(X_train[categoricals])
X_train = pd.DataFrame(encoder.transform(X_train[categoricals]).toarray(),
                         columns=encoder.get_feature_names(categoricals))
X_test = pd.DataFrame(encoder.transform(X_test[categoricals]).toarray(),
                        columns=encoder.get_feature_names(categoricals))

#print(encoder.categories_)
#[print(col, end=', ') for col in ohe.columns];
#ohe.head()

In [10]:
dt = DecisionTreeClassifier(random_state=25565, min_samples_leaf=30)
dt.fit(X_train, y_train)
#dt.score(X_train, y_train), dt.score(X_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=25565, splitter='best')

In [12]:
# RandomForestClassifier, BaggingClassifier

In [13]:
gb = GradientBoostingClassifier(random_state=25565, min_samples_leaf=30)
gb.fit(X_train, y_train)
gb.score(X_train, y_train), gb.score(X_test, y_test)

(0.9533360622185837, 0.9623567921440261)

In [14]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)
bg.score(X_train, y_train), gb.score(X_test, y_test)

(0.9959066721244372, 0.9623567921440261)

In [15]:
rf = RandomForestClassifier(random_state=25565, n_estimators=100)
rf.fit(X_train, y_train)
rf.score(X_train, y_train), rf.score(X_test, y_test)

(0.9963160049119935, 0.9963175122749591)

In [11]:
# col_1 = unsafe, col_2 = safe
dt.predict_proba(X_train)

array([[0.88306452, 0.11693548],
       [1.        , 0.        ],
       [0.88306452, 0.11693548],
       ...,
       [1.        , 0.        ],
       [0.71028037, 0.28971963],
       [1.        , 0.        ]])

In [33]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def score(model, X_test, y_test, clf_thres):
    y_pred = model.predict_proba(X_test)
    y_pred = np.array([0 if y[1] < clf_thres else 1 for y in y_pred])
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred))
    print('F1 score:', f1_score(y_test, y_pred))
    print('Recall:', recall_score(y_test, y_pred))

In [36]:
dt.score(X_test, y_test)

0.9549918166939444

In [39]:
score(dt, X_test, y_test, .5)

Accuracy: 0.9549918166939444
Precision: 0.7217391304347827
F1 score: 0.6014492753623188
Recall: 0.515527950310559


In [40]:
score(dt, X_test, y_test, .1)

Accuracy: 0.8203764320785597
Precision: 0.23473282442748092
F1 score: 0.35912408759124087
Recall: 0.7639751552795031


In [41]:
score(dt, X_test, y_test, .3)

Accuracy: 0.9549918166939444
Precision: 0.7217391304347827
F1 score: 0.6014492753623188
Recall: 0.515527950310559


In [42]:
score(dt, X_test, y_test, .9)

Accuracy: 0.9509001636661211
Precision: 1.0
F1 score: 0.4059405940594059
Recall: 0.2546583850931677
